<a href="https://colab.research.google.com/github/irishka-learns/freecodecamp/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [6]:
import re
import string

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [10]:
train_data = pd.read_csv(train_file_path, sep='\t', names=['label', 'text'], header=None)
test_data = pd.read_csv(test_file_path, sep='\t', names=['label', 'text'], header=None)
train_data.head()

,label,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [11]:
label_mapping = {'ham': 0, 'spam': 1}
train_data['label'] = train_data['label'].map(label_mapping)
test_data['label'] = test_data['label'].map(label_mapping)
train_data.head()

,label,text
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [12]:
label_train = train_data.pop('label')
label_test = test_data.pop('label')

In [15]:
train_data = train_data.values
label_train = label_train.values
test_data = test_data.values
label_test = label_test.values

In [26]:
#def standardize_text(text):
#  lowercase = tf.strings.lower(text)
#  return tf.strings.regex_replace(lowercase, [%s] % re.escape(string.punctuation), '')

vc = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=10000, output_mode='int', output_sequence_length=250)

vc.adapt(train_data)

In [54]:
model = keras.Sequential([
                          keras.layers.Input(shape=(1,), dtype=tf.string),
                          vc,
                          keras.layers.Embedding(10001, 32),
                          keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences = True)),
                          keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),                          
                          keras.layers.Dense(32, activation='relu'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_4 (TextVe (None, 250)               0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 250, 32)           320032    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 250, 64)           16640     
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 32)                10368     
_________________________________________________________________
dense_20 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_18 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)               

In [55]:
model.fit(x= train_data, y=label_train, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 41s 330ms/step - loss: 0.4140 - accuracy: 0.8779 - val_loss: 0.0935 - val_accuracy: 0.9821
Epoch 2/10
105/105 [==============================] - 34s 324ms/step - loss: 0.0672 - accuracy: 0.9867 - val_loss: 0.0769 - val_accuracy: 0.9713
Epoch 3/10
105/105 [==============================] - 35s 333ms/step - loss: 0.0185 - accuracy: 0.9953 - val_loss: 0.0687 - val_accuracy: 0.9844
Epoch 4/10
105/105 [==============================] - 37s 349ms/step - loss: 0.0037 - accuracy: 0.9994 - val_loss: 0.0853 - val_accuracy: 0.9797
Epoch 5/10
105/105 [==============================] - 37s 351ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.0823 - val_accuracy: 0.9868
Epoch 6/10
105/105 [==============================] - 38s 359ms/step - loss: 5.2504e-04 - accuracy: 0.9999 - val_loss: 0.1060 - val_accuracy: 0.9809
Epoch 7/10
105/105 [==============================] - 37s 355ms/step - loss: 4.1534e-04 - accuracy: 1.0000 - val_loss: 0.0980 

In [56]:
model.evaluate(x= test_data, y= label_test)

44/44 [==============================] - 3s 68ms/step - loss: 0.1152 - accuracy: 0.9784


[0.1152326837182045, 0.9784482717514038]

In [58]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  probability = model.predict(np.array([pred_text]))
  prediction = [probability[0][0]]
  if probability > 0.5:
    prediction.append('spam')
  else:
    prediction.append('ham')
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[2.3767012e-05, 'ham']


In [60]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
